# Open Private Join and Activation Cryptography Overview

This document provides an overview of the cryptographic primitives, operations, workflows found in the IAB Tech Lab [Open Private Join and Activation](https://iabtechlab.com/datacleanrooms/) (OPJA) standard. This is accomplished by presenting high-level template implementations (written using Python 3) of encryption/decryption operations, protocol participants, and (for context) some data management workflows. The underlying cryptographic primitives required for those workflows are invoked via the interface provided by the [cryptography.io](https://cryptography.io) Python library.

The workflow implementations in this document are only *illustrations* that can serve as a guide and aid in understanding the OPJA standard (*e.g.*, when assembling a prototype, development, or production implementations of a component that must conform to OPJA). These illustrations do not exhaustively acknowledge or address all security, privacy, performance, scalability, software engineering, and information technology issues that may be considered in production implementations.

The definitions and implementatinos found in this document are organized according to common encapsulation, modularity, and reuse principles drawn from the practice of software engineering. Thus, the  order in which they appear may not match the OPJA standard.

## Dependencies

This document requires Python 3.11 and is designed to be viewed and executed using [Jupyter Notebook](https://jupyter.org/). The document also relies on a few additional dependencies. All required dependencies can be found in the accompanying `requirements.txt` file.

The Python class and function definitions in this document are annotated with their types.

In [1]:
from __future__ import annotations
from typing import Optional, Tuple, Sequence

A number of built-in libraries are used throughout this document.

In [2]:
import os
import base64
import collections
import uuid

Cryptographic primitives within this document are invoked via the interface provided by the [cryptography.io](https://cryptography.io) Python library. In many environments, installing the library should be sufficient. However, in some cases, there may be a mismatch between the latest [cryptography.io](https://cryptography.io) release and the particular version of OpenSSL (or equivalent) that is installed or against which the installed version of Python is compiled.

In [3]:
import cryptography

## Common Operations

Basic implementations of AES-128 GCM encryption and decryption functions are presented below, based on an [example](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.modes.GCM) found in the [cryptography.io](https://cryptography.io) documentation.

In [4]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey

def aes_gcm_encrypt(key, plaintext, associated_data, nonce):
    """
    Encrypt a plaintext (coupled with unencrypted associated data and
    using the specified nonce); return the ciphertext and accompanying tag.
    """
    # Construct an AES-GCM `Cipher` object with the given `key` and a
    # randomly generated `nonce`.
    encryptor = Cipher(algorithms.AES(key), modes.GCM(nonce)).encryptor()

    # `associated_data` will be authenticated but not encrypted,
    # it must also be passed in on decryption.
    encryptor.authenticate_additional_data(associated_data)

    # Encrypt the plaintext and get the associated ciphertext.
    # GCM does not require padding.
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()

    return (ciphertext, encryptor.tag)

def aes_gcm_decrypt(key, associated_data, ciphertext, tag, nonce):
    """
    Decrypt a ciphertext (coupled with the associated data, tag, and
    nonce that were involved in the original encryption of the ciphertext).
    """
    # Construct a `Cipher` object, with the `key`, `nonce`, and
    # the GCM `tag` used for authenticating the message.
    decryptor = Cipher(algorithms.AES(key), modes.GCM(nonce, tag)).decryptor()

    # Put `associated_data` back or the `tag` will fail to verify
    # when the decryptor is finalized.
    decryptor.authenticate_additional_data(associated_data)

    # Decryption gets us the authenticated plaintext.
    # If the tag does not match an InvalidTag exception will be raised.
    return decryptor.update(ciphertext) + decryptor.finalize()

Below is a simple test of the two functions defined above.

In [5]:
private_key = X25519PrivateKey.generate()
public_key = private_key.public_key()
shared_key = private_key.exchange(public_key)
nonce = bytes(12)
(ciphertext, tag) = aes_gcm_encrypt(shared_key, b'message', b'assoc data', nonce)
assert b'message' == aes_gcm_decrypt(shared_key, b'assoc data', ciphertext, tag, nonce)

## Activation Protocol

### Label Encryption and Decryption

The activation protocol involves the preparation and delivery of *encrypted labels*. Below are minimal implementations of functions for encrypting and decrypting an ordered collection of labels. These implementations adhere to the OPJA specification.

In [6]:
def encrypt_labels(
        key: bytes,
        match_transaction_id: bytes,
        ls: Sequence[bool]
    ) -> Sequence[str]:
    """
    Encrypt a sequence of boolean labels and return the sequence of
    ciphertexts.
    """
    es = []
    nonce_base = os.urandom(12)

    for (i, l) in enumerate(ls):
        b = bytes([255 if l else 0])
        nonce = bytes([x ^ y for (x, y) in zip(i.to_bytes(12, 'big'), nonce_base)])
        (c, t) = aes_gcm_encrypt(
            key,
            b,
            match_transaction_id,
            nonce
        )
        e = base64.standard_b64encode(nonce + c + t).decode()
        es.append(e)

    return es

def decrypt_labels(
        key: bytes,
        match_transaction_id: bytes,
        cs: Sequence[str]
    ) -> Sequence[bool]:
    """
    Decrypt a sequence of encrypted label ciphertexts and return the
    original labels.
    """
    ls = []

    for (i, c) in enumerate(cs):
        raw = base64.standard_b64decode(c.encode())
        (nonce, c, tag) = (raw[:12], raw[12:-16], raw[-16:])
        l = 255 == aes_gcm_decrypt(key, match_transaction_id, c, tag, nonce)[0]
        ls.append(l)

    return ls

Below is a simple test of the functions defined above.

In [7]:
ls = [True, False, True, False]
match_transaction_id = b'1234567890'
es = encrypt_labels(shared_key, match_transaction_id, ls)
ls_ = decrypt_labels(shared_key, match_transaction_id, es)
assert ls == ls_

### Participants

All participants (publishers, advertisers, matching systems, DSPs, and SSPs) must have the capacity to perform some basic cryptographic key management operations. The class definition below includes methods corresponding to these operations.

In [8]:
class Participant:
    """
    Functionalities common to all participants.
    """
    def __init__(self: Participant, identifier: Optional[str] = None):
        """
        Each participant has a unique identifier
        """
        self.identifier = uuid.uuid4() if identifier is None else identifier

    def generate_key_pair(self: Participant) -> Tuple[bytes, bytes]:
        """
        Generate an individual public-private key pair.
        """
        key_private = X25519PrivateKey.generate()
        key_public = key_private.public_key()
        return (key_public, key_private)

    def initial_key_pairs(self: Participant):
        """
        Prepare an ordered collection of five public-private key pairs.
        The "first" key is at the right-most end of the ordered collection.
        """
        self.key_pairs = collections.deque([self.generate_key_pair() for _ in range(5)])
        
    def rotate_key_pairs(self: Participant):
        """
        Rotate the key pairs by removing the "last" (left-most end) key pair
        in the collection and adding a newly generated key pair.
        """
        self.key_pairs.popleft()
        self.key_pairs.append(self.generate_key_pair())

    def first_key_pair(self: Participant) -> Tuple[bytes, bytes]:
        """
        Return the newest key pair (*i.e.*, right-most end).
        """
        return self.key_pairs[-1]

    def first_key_public(self: Participant) -> bytes:
        """
        Return the public key from the newest key pair (*i.e.*, right-most end).
        """
        return self.first_key_pair()[0]

    def first_key_private(self: Participant) -> bytes:
        """
        Return the private key from the newest key pair (*i.e.*, right-most end).
        """
        return self.first_key_pair()[1]


### Example Workflow

Below is a sketch (using the functions and classes defined above) of a basic activation workflow between a matching system and a DSP.

In [9]:
class MatchingSystem(Participant):
    def encrypt_labels_for(
            self: MatchingSystem,
            participant: Participant,
            match_transaction_id: bytes,
            ls: Sequence[bool]
        ) -> Sequence[str]:
        key_shared = self.first_key_private().exchange(participant.first_key_public())
        return encrypt_labels(key_shared, match_transaction_id, ls)

class ActivationSystem(Participant):
    def decrypt_labels_from(
            self: ActivationSystem,
            participant: Participant,
            match_transaction_id: bytes,
            es: Sequence[str]
        ) -> Sequence[bool]:
        key_shared = self.first_key_private().exchange(participant.first_key_public())
        return decrypt_labels(key_shared, match_transaction_id, es)

In [10]:
matching_system = MatchingSystem()
matching_system.initial_key_pairs()

dsp = ActivationSystem()
dsp.initial_key_pairs()

ls = [True, False, True, False]
match_transaction_id = b'1234567890'
es = matching_system.encrypt_labels_for(dsp, match_transaction_id, ls)
ls_ = dsp.decrypt_labels_from(matching_system, match_transaction_id, es)
assert ls == ls_

In [ ]:
# End of file.